In [4]:
!apt install -y curl
!curl -L -o ./dynamically-generated-hate-speech-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/usharengaraju/dynamically-generated-hate-speech-dataset

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  krb5-locales libbrotli1 libcurl4t64 libgssapi-krb5-2 libk5crypto3
  libkeyutils1 libkrb5-3 libkrb5support0 libldap-common libldap2 libnghttp2-14
  libpsl5t64 librtmp1 libsasl2-2 libsasl2-modules libsasl2-modules-db libssh-4
  publicsuffix
Suggested packages:
  krb5-doc krb5-user libsasl2-modules-gssapi-mit
  | libsasl2-modules-gssapi-heimdal libsasl2-modules-ldap libsasl2-modules-otp
  libsasl2-modules-sql
The following NEW packages will be installed:
  curl krb5-locales libbrotli1 libcurl4t64 libgssapi-krb5-2 libk5crypto3
  libkeyutils1 libkrb5-3 libkrb5support0 libldap-common libldap2 libnghttp2-14
  libpsl5t64 librtmp1 libsasl2-2 libsasl2-modules libsasl2-modules-db libssh-4
  publicsuffix
0 upgraded, 19 newly installed, 0 to remove and 0 not upgraded.
Need to get 2406 kB of archives.
After this operation, 6886 kB of additional disk 

In [9]:
!apt install zip unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 350 kB of archives.
After this operation, 933 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu noble-updates/main amd64 unzip amd64 6.0-28ubuntu4.1 [174 kB]
Get:2 http://archive.ubuntu.com/ubuntu noble-updates/main amd64 zip amd64 3.0-13ubuntu0.2 [176 kB]
Fetched 350 kB in 2s (184 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 10140 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-28ubuntu4.1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######....................................................] 8Unpacking unzip (6.0-28ubun

In [11]:
!unzip ./dynamically-generated-hate-speech-dataset.zip

Archive:  ./dynamically-generated-hate-speech-dataset.zip
  inflating: 2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv  
  inflating: 2020-12-31-DynamicallyGeneratedHateDataset-targets-v0.1.csv  


In [74]:
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pprint import pprint
from torch.nn.functional import softmax
import numpy as np
from tqdm.notebook import tqdm
import torch

In [75]:
ds1=load_dataset("toxigen/toxigen-data", "annotated", split="train")
ds2=load_dataset("toxigen/toxigen-data", "annotated", split="test")
ds=concatenate_datasets([ds1, ds2])

In [76]:
ds

Dataset({
    features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'],
    num_rows: 9900
})

In [77]:
def introduce_toxic_labels(ex_batch):
    return {
        "text": [ex for ex in ex_batch["text"]],
        "toxic": [1 if ex>2.5 else 0 for ex in ex_batch["toxicity_human"]]
    }

In [78]:
ds=ds.map(
    introduce_toxic_labels,
    batched=True,
    remove_columns=ds.column_names
)

Map:   0%|          | 0/9900 [00:00<?, ? examples/s]

In [79]:
checkpoint="IMSyPP/hate_speech_en"
model=AutoModelForSequenceClassification.from_pretrained(checkpoint, device_map="auto")
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [82]:
def test_model(ds, tokenizer, model):
    true_positive=0
    true_negative=0
    false_positive=0
    false_negative=0
    total_count=0
    
    model.eval()

    with torch.inference_mode():
        for ex in tqdm(ds):
            tokens=tokenizer(ex["text"], return_tensors="pt").to(model.device)
            op=model(**tokens)
            logits=op.logits
            prob=softmax(logits, dim=-1)
            pred=np.argmax(prob.cpu().detach().numpy(), axis=-1)[0]

            if(pred==0 and ex["toxic"]==0):
                true_negative+=1
            elif (pred==0 and ex["toxic"]==1):
                false_negative+=1
            elif (pred!=0 and ex["toxic"]==0):
                false_positive+=1
            elif (pred!=0 and ex["toxic"]==1):
                true_positive+=1
            total_count+=1

    precision=true_positive/(true_positive+false_positive)
    recall=true_positive/(true_positive+false_negative)
    f1=(2*precision*recall)/(precision+recall)
    accuracy=(true_positive+true_negative)/(true_positive+false_positive+true_negative+false_negative)
    
    return {
        "no of examples tested": total_count,
        "accuracy": f"{accuracy*100:4f}%",
        "precision": f"{precision*100:4f}%",
        "recall": f"{recall*100:4f}%",
        "f1 score": f"{f1*100:4f}%"
    }

In [83]:
results=test_model(ds, tokenizer,  model)

  0%|          | 0/9900 [00:00<?, ?it/s]

In [84]:
results

{'no of examples tested': 9900,
 'accuracy': '68.727273%',
 'precision': '74.988914%',
 'recall': '40.042624%',
 'f1 score': '52.207471%'}